In [5]:
import pandas as pd
import numpy as np
import mysql.connector
import os
from mysql.connector import Error
from datetime import datetime, date, timedelta
import ConeccionesSucursales as cs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [6]:
#Definimos la ruta en donde queremos guardar los archivos
ruta_guardar = '/home/estadistico/Documents/Erick/Reportes diarios'
ruta_asig_baz = '/home/estadistico/Documents/Erick/Banco Azteca/Asignacion csv'
ruta = '/home/estadistico/Documents/Erick'
mes = 20210101
mes2 = 'Enero2021'
mes3 = pd.to_datetime('01/01/2021',dayfirst=True)
fin_mes = pd.to_datetime('31/01/2021',dayfirst=True)
prim_sem_mes = 1

In [7]:
#Agregamos las variables a ocupar
servidor = '192.168.15.12'
puerto = int('3306')
usuario = 'estadisticas'
contraseña = 'estadisticas8474'
base = 'procesos_externos'

In [8]:
#Lectura pagos Bradesco
book = os.listdir(''+ruta+'/'+str(mes2)+'/Bradesco/Pagos/')
pagos_brad = pd.DataFrame()
for i in book:
    pagos_new = pd.read_excel(''+ruta+'/'+str(mes2)+'/Bradesco/Pagos/'+i+'')
    pagos_new = pagos_new.iloc[:,0:9]
    pagos_brad = pd.concat([pagos_brad,pagos_new],ignore_index=True)
    pagos_brad = pagos_brad.drop_duplicates()
pagos_brad = pd.pivot_table(pagos_brad,index=['NUMERO_TARJETA','FECHA_PAGO','TIPO_ASIGANCION'],values=['MONTO'],aggfunc=np.sum)
pagos_brad = pd.DataFrame(pagos_brad.to_records())
pagos_brad = pagos_brad.loc[pagos_brad['FECHA_PAGO']>=mes3]

In [9]:
book =  os.listdir(''+ruta+'/'+str(mes2)+'/Liverpool/Pagos/')
sheet_liv = ['SA002REM','SA001REM','DKN04REM','DKN02REM','DKN00REM']
pagos_liv = pd.DataFrame()
pagos_liv_new = pd.DataFrame()
for i in book:
    for j in sheet_liv:
        pagos_new = pd.read_excel(''+ruta+'/'+str(mes2)+'/Liverpool/Pagos/'+i+'',sheet_name = str(j),converters={'Cuenta':str})
        pagos_new.columns = ['Id','Cuenta','Fecha Prom','Fecha Pago','Monto','Posteo']
        pagos_liv_new = pd.concat([pagos_liv_new,pagos_new],ignore_index=True)
        pagos_liv_new = pagos_liv_new.iloc[:,0:6]
    pagos_liv = pd.concat([pagos_liv,pagos_liv_new],ignore_index=True)
    pagos_liv = pagos_liv.drop_duplicates()
pagos_liv['Fecha Pago'] = pd.to_datetime(pagos_liv['Fecha Pago'],dayfirst=True)
pagos_liv = pd.pivot_table(pagos_liv,index=['Id','Cuenta','Fecha Pago','Fecha Prom'],values=['Monto'],aggfunc=np.sum)
pagos_liv = pd.DataFrame(pagos_liv.to_records())
pagos_liv = pagos_liv.loc[pagos_liv['Fecha Pago']>=mes3]

In [10]:
#Lectura pagos Credifiel
book = os.listdir(''+ruta+'/'+str(mes2)+'/Credifiel/Pagos/')
pagos_cred = pd.DataFrame()
for j in book:
    pagos_new = pd.read_excel(''+ruta+'/'+str(mes2)+'/Credifiel/Pagos/'+j+'')
    pagos_cred = pd.concat([pagos_cred,pagos_new],ignore_index=True)
#pagos_cred = pagos_cred.iloc[:,[12,1,8]]
pagos_cred2 = pd.pivot_table(pagos_cred,index=['ID','FECHA'],values=['IMPORTE'],aggfunc=np.sum)
pagos_cred2 = pd.DataFrame(pagos_cred2.to_records())
pagos_cred2 = pagos_cred2.loc[pagos_cred2['FECHA']>=mes3]

In [11]:
#Lectura pagos Banco Azteca
#sheet_baz = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/')
#pagos_baz = pd.DataFrame()
#for k in sheet_baz:
#    pagos_baz_new = pd.read_csv(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/'+k+'')
#    pagos_baz = pd.concat([pagos_baz,pagos_baz_new],ignore_index=True)
#pagos_baz = pagos_baz.drop_duplicates()
#pagos_baz = pagos_baz.loc[pagos_baz['FNMONTOPAGADO']>0]
#pagos_baz['FDFECHAABONO'] = pd.to_datetime(pagos_baz['FDFECHAABONO'],dayfirst=True)
#pagos_baz = pagos_baz.loc[pagos_baz['FDFECHAABONO']>=mes3]
#pagos_baz2 = pd.pivot_table(pagos_baz,index=['CAMPANAID','FDFECHAABONO','FIIDPERIODO','FNSCOMPROMISO','FCEMPNUMCORTE','FNMONTOPROMETIDO'],values=['FNMONTOPAGADO'],aggfunc=np.sum)
#pagos_baz2 = pd.DataFrame(pagos_baz2.to_records())

In [12]:
#Leemos facturacion Banco Azteca
#sheet_baz2 = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Facturacion/')
#fact_baz = pd.DataFrame()
#for k in sheet_baz2:
#    fact_baz_new = pd.read_csv(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Facturacion/'+k+'',encoding='latin-1')
#    fact_baz = pd.concat([fact_baz,fact_baz_new],ignore_index=True)
#fact_baz = fact_baz.drop_duplicates()
#fact_baz = fact_baz.loc[fact_baz['Número Semana']>=prim_sem_mes]
#fact_baz['Final de Semana'] = pd.to_datetime(fact_baz['Final de Semana'],dayfirst=True)
#fact_baz['Inicio de Semana'] = pd.to_datetime(fact_baz['Inicio de Semana'],dayfirst=True)

In [13]:
#Asignamos valores a los parametros \n"
today = date.today().strftime('%Y%m%d')
now = datetime.now().strftime('%d-%m-%Y %H-%M')
hoy = np.datetime64('today')
ayer = np.datetime64(date.today() - timedelta(1))
today2 = date.today().strftime('%d-%m-%Y')

In [14]:
#Hacemos la conexion con el servidor\n",
try:
    conn = mysql.connector.connect(user=usuario,
                               password=contraseña,
                               host=servidor,
                               port=3306,
                               database=base)
    conn.set_charset_collation('latin1')
except mysql.Error as e:
    print("Failed to execute stored procedure: {}".format(error))

cursor = conn.cursor()

In [15]:
#Hacemos la asignacion de cada una de las metas para cambiar de arancel sin tomar el 0 en cuenta
c04pv1, arancel_4pv1 = 1204704, .07
c04pv2, arancel_4pv2 = 1807056, .09
c04pv3, arancel_4pv3 = 2409407, .12
c05pv1, arancel_5pv1 = 662269, .09
c05pv2, arancel_5pv2 = 993403, .10
c05pv3, arancel_5pv3 = 1324538, .14
c06pv1, arancel_6pv1 = 807172, .10
c06pv2, arancel_6pv2 = 1210759, .11
c06pv3, arancel_6pv3 = 1614345, .16
a1, arancel_a1 = 260190, .27
a2, arancel_a2 = 390285, .30
a3, arancel_a3 = 520381, .33
b1, arancel_b1 = 215083, .25
b2, arancel_b2 = 322624, .28
b3, arancel_b3 = 430165, .31
c11, arancel_c11 = 1430989, .17
c12, arancel_c12 = 2146483, .20
c13, arancel_c13 = 2861978, .23
c21, arancel_c21 = 626634, .19
c22, arancel_c22 = 939951, .22
c23, arancel_c23 = 1253267, .25
c31, arancel_c31 = 246425, .21
c32, arancel_c32 = 369637, .24
c33, arancel_c33 = 492849, .27
c41, arancel_c41 = 123529, .23
c42, arancel_c42 = 185294, .26
c43, arancel_c43 = 247059, .29

In [16]:
#Funcion para hacer la consulta sql, ya sea de gestion o de asignacion en donde requerimos tres parametros, la tabla de donde vamos a sacar la consulta, en caso de asignacin requerimos los otros dos parametros fecha inicial y fecha final
def consulta_sql(table,from_date=None,to_date=None):
    if from_date is None:
        sql_mes = cursor.callproc(table,)
        for result in cursor.stored_results():
            sql_mes = pd.DataFrame(result.fetchall())
    else:
        sql_mes = cursor.callproc(table, [from_date,to_date])
        for result in cursor.stored_results():
            sql_mes = pd.DataFrame(result.fetchall())
    sql_mes = sql_mes.drop_duplicates()
    return sql_mes

In [17]:
#Funcion para transformar varias columnas a numero
def transform_numeric(df,cols):
    return df[cols].apply(pd.to_numeric, errors='coerce')

#Funcion que convierte a dataframe y rellena espacios con 0
def to_dataframe(df):
    df = pd.DataFrame(df.to_records())
    df.fillna(0,inplace=True)
    return df

# Reporte Liverpool

In [18]:
#Hacemos la consulta referente a las gestiones de Liverpool
gestion_liv_mes = cs.query_naucalpan('liverpool_rpt_gestiones_detallado',mes,today)
gestion_liv_mes.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','fecha_promesa','monto_promesa','comentarios']
gestion_liv_mes = gestion_liv_mes.loc[gestion_liv_mes['dictamen']=='PROMESA']
gestion_liv_mes['anio'] = [int(x[:4]) for x in gestion_liv_mes['fecha_promesa']]
gestion_liv_mes = gestion_liv_mes.loc[(gestion_liv_mes['anio']>2010)&(gestion_liv_mes['anio']<2100)]
gestion_liv_mes['fecha_promesa'] = pd.to_datetime(gestion_liv_mes['fecha_promesa'],dayfirst=False)

In [19]:
#Asignacion Liverpool
asignacion_liv = pd.read_csv(''+ruta_pagos+'/Liverpool/Asignacion/Asignaciones csv/Asignacion '+mes2+'.csv',converters={'# CUENTA':str})
asignacion_liv = asignacion_liv[['ID AGENCIA', '# CUENTA']]
asignacion_liv.columns = ['division','credito']
asignacion_liv.head(1)

,division,credito
0,SA002REM,0001400000007731973


In [20]:
#Hacemos el consolidado
consolidado_liv_mes = pd.merge(gestion_liv_mes,asignacion_liv,how='left',on=['credito'])
cols = ['monto_promesa']
consolidado_liv_mes[cols] = consolidado_liv_mes[cols].apply(pd.to_numeric, errors='coerce')
consolidado_liv_mes = consolidado_liv_mes[(consolidado_liv_mes['monto_promesa']>50)]
consolidado_liv_mes = consolidado_liv_mes[(consolidado_liv_mes['monto_promesa']<500000)]
consolidado_liv_mes['division'].fillna('COBRANZA',inplace=True)
consolidado_liv_mes['division'] = consolidado_liv_mes['division'].replace('','COBRANZA')
consolidado_liv_mes = consolidado_liv_mes.drop_duplicates(subset = ['usuario','credito','fecha_promesa','monto_promesa'])

In [21]:
productos = [('DKN00REM','pren'),('DKN02REM','REM2'),('DKN04REM','REM4'),('SA001REM','SB02'),('SA002REM','REM9')]
productos = pd.DataFrame(productos)
productos.columns = ['id','division']
consolidado_liv_mes = pd.merge(consolidado_liv_mes,productos,how='left',on=['division'])

In [23]:
#Asignamos los aranceles de liverpool
aranceles = [('DKN00REM',0.28),('DKN02REM',0.4),('DKN04REM',0.5),('SA001REM',0.15),('SA002REM',0.16)]
aranceles = pd.DataFrame(aranceles)
aranceles.columns = ['Id','Arancel']

#Agregamos aranceles al consolidado
pagos_liv3 = pd.merge(pagos_liv,aranceles,how='left',on='Id')
pagos_liv3['PagoArancel'] = [x*y for x,y in zip(pagos_liv3['Monto'],pagos_liv3['Arancel'])]

In [24]:
#Unimos el consolidado con los pagos 
consolidado_pagos_liv = pd.merge(consolidado_liv_mes,pagos_liv3,how='left',left_on=['credito'],right_on=['Cuenta'])
consolidado_pagos_liv = consolidado_pagos_liv.drop_duplicates(subset = ['credito','Monto','usuario','fecha_promesa'])
consolidado_pagos_liv['Fecha Pago'] = pd.to_datetime(consolidado_pagos_liv['Fecha Pago'],dayfirst=True)
consolidado_pagos_liv['dia_prom'] = consolidado_pagos_liv['fecha_promesa'].dt.day
consolidado_pagos_liv['dia_pago'] = consolidado_pagos_liv['Fecha Pago'].dt.day
cols = ['dia_prom','dia_pago','Monto']
consolidado_pagos_liv[cols] = consolidado_pagos_liv[cols].apply(pd.to_numeric, errors='coerce',downcast='integer')
consolidado_pagos_liv['dia_pago'].fillna(0,inplace=True)
try:
    consolidado_pagos_liv['dia_pago'] = [int(x) for x in consolidado_pagos_liv['dia_pago']]
except: consolidado_pagos_liv['dia_pago'] = consolidado_pagos_liv['dia_pago']

In [25]:
#Condicionamos a los gestores cuyas promesas y pagos estan en un plazo de 3 dias
consolidado_pagos_liv['PagoReal'] = [np.where((z < x+pd.Timedelta(3, unit='D')) & ((z > x-pd.Timedelta(3, unit='D') )), y, 0) for x,y,z in zip(consolidado_pagos_liv['fecha_promesa'],consolidado_pagos_liv['Monto'],consolidado_pagos_liv['Fecha Pago'])]
consolidado_pagos_liv['PagoReal'] = [x.item() for x in consolidado_pagos_liv['PagoReal']]

In [26]:
consolidado_pagos_liv['MontoArancel'] = [x*y for x,y in zip(consolidado_pagos_liv['PagoReal'],consolidado_pagos_liv['Arancel'])]

In [27]:
#Obtenemos datos de los gestores de Liverpool
promesas_mes_liv = pd.pivot_table(consolidado_pagos_liv,index=['division'],values=['monto_promesa'],aggfunc=['count',np.sum],margins=True,dropna=False)
promesas_mes_liv = pd.DataFrame(promesas_mes_liv.to_records())
promesas_mes_liv.columns = ['Division','NumeroPromesas','SumaPromesas']
promesas_mes_liv.fillna(0,inplace=True)
promesas_mes_liv

,Division,NumeroPromesas,SumaPromesas
0,COBRANZA,4,31128.28
1,DKN00REM,179,749282.74
2,DKN02REM,152,669093.44
3,DKN04REM,78,261983.37
4,SA001REM,170,272277.24
5,SA002REM,10,21957.00
6,All,593,2005722.07


In [28]:
#Obtenemos las promesas restantes a partir del día de hoy
restantes_liv = consolidado_pagos_liv.loc[(consolidado_pagos_liv['fecha_promesa']>=ayer)&(consolidado_pagos_liv['fecha_promesa']<=fin_mes)]
promesas_restantes_liv = pd.pivot_table(restantes_liv,index=['division'],values=['monto_promesa'],aggfunc=[np.sum],dropna=False)
promesas_restantes_liv = pd.DataFrame(promesas_restantes_liv.to_records())
promesas_restantes_liv.columns = ['Division','PromesasRestantes']
promesas_mes_liv.fillna(0,inplace=True)
promesas_restantes_liv

,Division,PromesasRestantes
0,DKN00REM,192472.87
1,DKN02REM,151329.00
2,DKN04REM,72174.00
3,SA001REM,81950.10


In [30]:
#Total pagos Liver registrados
pagos_totales_liv = pd.pivot_table(pagos_liv3,index=['Id'],values=['Monto','PagoArancel'],aggfunc=[np.sum,'count'],dropna=False)
pagos_totales_liv = pd.DataFrame(pagos_totales_liv.to_records())
pagos_totales_liv = pagos_totales_liv.iloc[:,[0,3,1,2]]
pagos_totales_liv.columns = ['Division','NumeroPagos','SumaPagos','SumaFacturacion']
pagos_totales_liv.fillna(0,inplace=True)
pagos_totales_liv['FechaFact'] = max(pagos_liv['Fecha Pago'])
pagos_totales_liv['FechaFact'] = pd.to_datetime(pagos_totales_liv['FechaFact'],dayfirst=True)
pagos_totales_liv

,Division,NumeroPagos,SumaPagos,SumaFacturacion,FechaFact
0,DKN00REM,103,399556.00,111875.680,2021-01-17
1,DKN02REM,97,357302.99,142921.196,2021-01-17
2,DKN04REM,50,180155.37,90077.685,2021-01-17
3,SA001REM,107,153899.98,23084.997,2021-01-17
4,SA002REM,9,16701.00,2672.160,2021-01-17


In [31]:
final_liv = pd.merge(pagos_totales_liv,promesas_mes_liv,how='left',on=['Division'])
final_liv = pd.merge(final_liv,promesas_restantes_liv,how='left',on=['Division'])
final_liv.insert(loc=0,column='Cliente',value='Liverpool')
final_liv

,Cliente,Division,NumeroPagos,SumaPagos,SumaFacturacion,FechaFact,NumeroPromesas,SumaPromesas,PromesasRestantes
0,Liverpool,DKN00REM,103,399556.00,111875.680,2021-01-17,179,749282.74,192472.87
1,Liverpool,DKN02REM,97,357302.99,142921.196,2021-01-17,152,669093.44,151329.00
2,Liverpool,DKN04REM,50,180155.37,90077.685,2021-01-17,78,261983.37,72174.00
3,Liverpool,SA001REM,107,153899.98,23084.997,2021-01-17,170,272277.24,81950.10
4,Liverpool,SA002REM,9,16701.00,2672.160,2021-01-17,10,21957.00,NaN


# Reporte Bradesco

In [32]:
asignacion_brad_c0 = pd.read_csv(''+ruta+'/Bradesco/Asignacion_c0/CSV/ASIGNACION C0 '+mes2+'.csv')
asignacion_brad_c0 = asignacion_brad_c0.iloc[:,[0,2,3,8,10,11,14,25]]
asignacion_brad_extra = pd.read_csv(''+ruta+'/Bradesco/Asignacion_extra/CSV/Asignacion Extra '+mes2+'.csv',encoding= 'unicode_escape')
asignacion_brad_extra = asignacion_brad_extra.iloc[:,[0,2,3,5,9,25,26,28]]
asignacion_brad_extra.columns = ['CODIGO_ORG','NUMERO_CUENTA','NUMERO_TARJETA','RFC','ESTADO','SALDO_TOTAL','PAGO_DESCUENTO','PAGOSVENCIDOS']

asignacion_brad = pd.concat([asignacion_brad_c0,asignacion_brad_extra],ignore_index=True)

In [33]:
#Hacemos la consulta referente a las gestiones de Bradesco
gestion_brad_mes = cs.query_naucalpan('bradescard_rpt_gestiones_detallado',mes,today)
gestion_brad_mes.columns = ['folio_gestion','firma_id','unegocio_id','NUMERO_TARJETA','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','accion_resultado','fecha_promesa','monto_promesa','comentarios']
gestion_brad_mes = gestion_brad_mes.loc[gestion_brad_mes['dictamen']=='PROMESA']
gestion_brad_mes['NUMERO_TARJETA'] = pd.to_numeric(gestion_brad_mes['NUMERO_TARJETA'])
gestion_brad_mes['monto_promesa'] = pd.to_numeric(gestion_brad_mes['monto_promesa'])
gestion_brad_mes = gestion_brad_mes.drop_duplicates(subset=['NUMERO_TARJETA','usuario','fecha_promesa','monto_promesa'])

In [34]:
#Hacemos la union de asignacion y promesas
consolidado_brad_mes = pd.merge(gestion_brad_mes,asignacion_brad,how='left',on=['NUMERO_TARJETA'])
cols = ['monto_promesa']
consolidado_brad_mes[cols] = consolidado_brad_mes[cols].apply(pd.to_numeric, errors='coerce')
consolidado_brad_mes = consolidado_brad_mes[(consolidado_brad_mes['monto_promesa']>50)]
consolidado_brad_mes = consolidado_brad_mes[(consolidado_brad_mes['monto_promesa']<500000)]
#consolidado_brad_mes.fillna('COBRANZA',inplace=True)
consolidado_brad_mes['NUMERO_TARJETA'] = pd.to_numeric(consolidado_brad_mes['NUMERO_TARJETA'])
consolidado_brad_mes.head(1)

,folio_gestion,firma_id,unegocio_id,NUMERO_TARJETA,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,...,fecha_promesa,monto_promesa,comentarios,CODIGO_ORG,NUMERO_CUENTA,ESTADO,RFC,SALDO_TOTAL,PAGO_DESCUENTO,PAGOSVENCIDOS
0,9666010,1,3,5063290987214020,GONZALEZ FLORES NESTOR ADAN,4433211722,celular,2021-01-19 07:13:58,AGUILARMAVA,AGUILAR MACEDO VALERIA,...,2021-01-31,603.0,\t0443321172271 PROMESA DE PAGO DE LIQUIDACIO...,PLCC,5063290987214000,JALISCO,GOFN990307PP001,1722.41,1206.0,C0 4PV


In [67]:
#Obtenemos los pagos totales por division
pagos_totales_brad = pd.pivot_table(pagos_brad,index=['TIPO_ASIGANCION'],values=['MONTO'],aggfunc=[np.sum,'count'],margins=True,dropna=False)
pagos_totales_brad = pd.DataFrame(pagos_totales_brad.to_records())
pagos_totales_brad = pagos_totales_brad.iloc[:,[0,2,1]]
pagos_totales_brad.columns = ['Division','NumeroPagos','SumaPagos']
pagos_totales_brad.fillna(0,inplace=True)

In [68]:
C04PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 4PV', 'SumaPagos'].item()
C05PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 5PV', 'SumaPagos'].item()
C06PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 6PV', 'SumaPagos'].item()
A = pagos_totales_brad.loc[pagos_totales_brad['Division']=='A', 'SumaPagos'].item()
C1 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C1', 'SumaPagos'].item()
C2 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C2', 'SumaPagos'].item()
C3 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C3', 'SumaPagos'].item()
C4 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C4', 'SumaPagos'].item()
B = pagos_totales_brad.loc[pagos_totales_brad['Division']=='B', 'SumaPagos'].item()

In [70]:
def func(row):
    if row['TIPO_ASIGANCION'] == 'C0 4PV':
        return np.where((C04PV>0) & (C04PV<=c04pv1),arancel_4pv1,
                        np.where((C04PV>c04pv1) & (C04PV<=c04pv2),arancel_4pv2,
                                 np.where((C04PV>c04pv2) & (C04PV<=c04pv3),arancel_4pv3,arancel_4pv3+.03)))
    elif row['TIPO_ASIGANCION'] == 'C0 5PV':
        return np.where((C05PV>0) & (C05PV<=c05pv1),arancel_5pv1,
                        np.where((C05PV>c05pv1) & (C05PV<=c05pv2),arancel_5pv2,
                                 np.where((C05PV>c05pv2) & (C05PV<=c05pv3),arancel_5pv3,arancel_5pv3+.03)))
    elif row['TIPO_ASIGANCION'] == 'C0 6PV':
        return np.where((C06PV>0) & (C06PV<=c06pv1),arancel_6pv1,
                        np.where((C06PV>c06pv1) & (C06PV<=c06pv2),arancel_6pv2,
                                 np.where((C06PV>c06pv2) & (C06PV<=c06pv3),arancel_6pv3,arancel_6pv3+.03)))
    elif row['TIPO_ASIGANCION'] == 'A':
        return np.where((A>0) & (A<=a1),arancel_a1,
                        np.where((A>a1) & (A<=a2),arancel_a2,
                                 np.where((A>a2) & (A<=a3),arancel_a3,arancel_a3+.03)))
    elif row['TIPO_ASIGANCION'] == 'B':
        return np.where((B>0) & (B<=b1),arancel_b1,
                        np.where((B>b1) & (B<=b2),arancel_b2,
                                 np.where((B>b2) & (B<=b3),arancel_b3,arancel_b3+.03)))
    elif row['TIPO_ASIGANCION'] == 'C1':
        return np.where((C1>0) & (C1<=c11),arancel_c11,
                        np.where((C1>c11) & (C1<=c12),arancel_c12,
                                 np.where((C1>c12) & (C1<=c13),arancel_c13,arancel_c13+.03)))
    elif row['TIPO_ASIGANCION'] == 'C2':
        return np.where((C2>0) & (C2<=c21),arancel_c21,
                        np.where((C2>c21) & (C2<=c22),arancel_c22,
                                 np.where((C2>c22) & (C2<=c23),arancel_c23,arancel_c23+.03)))
    elif row['TIPO_ASIGANCION'] == 'C3':
        return np.where((C3>0) & (C3<=c31),arancel_c31,
                        np.where((C3>c31) & (C3<=c32),arancel_c32,
                                 np.where((C3>c32) & (C3<=c33),arancel_c33,arancel_c33+.03)))
    elif row['TIPO_ASIGANCION'] == 'C4':
        return np.where((C4>0) & (C4<=c41),arancel_c41,
                        np.where((C4>c41) & (C4<=c42),arancel_c42,
                                 np.where((C4>c42) & (C4<=c43),arancel_c43,arancel_c43+.03)))
    
    else:
        return 0

pagos_brad['Arancel'] = pagos_brad.apply(func, axis=1)

In [71]:
pagos_brad['Facturacion'] = [x*y for x,y in zip(pagos_brad['MONTO'],pagos_brad['Arancel'])]

In [72]:
#Hacemos la union consolidado y pagos
consoliado_pagos_brad = pd.merge(consolidado_brad_mes,pagos_brad,how='left',left_on=['NUMERO_TARJETA'],right_on=['NUMERO_TARJETA'])
try: consoliado_pagos_brad['fecha_promesa'] = pd.to_datetime(consoliado_pagos_brad['fecha_promesa'],dayfirst=False)
except: consoliado_pagos_brad['fecha_promesa'] = consoliado_pagos_brad['fecha_gestion']

In [73]:
#Condicionamos a los gestores cuyas promesas y pagos estan en un plazo de 3 dias
consoliado_pagos_brad['PagoUnico'] = [np.where((z < x+pd.Timedelta(4, unit='D')) & ((z > x-pd.Timedelta(3, unit='D') )), y, 0) for x,y,z in zip(consoliado_pagos_brad['fecha_promesa'],consoliado_pagos_brad['MONTO'],consoliado_pagos_brad['FECHA_PAGO'])]
consoliado_pagos_brad['PagoUnico'] = [x.item() for x in consoliado_pagos_brad['PagoUnico']]

In [74]:
consoliado_pagos_brad['PagoReal'] = [x*y for x,y in zip(consoliado_pagos_brad['PagoUnico'],consoliado_pagos_brad['Arancel'])]
consoliado_pagos_brad2 = consoliado_pagos_brad.drop_duplicates(subset=['NUMERO_CUENTA','MONTO','FECHA_PAGO','usuario'])

In [75]:
#Obtenemos datos de las promesas de Bradesco
prom_brad_div_mes = pd.pivot_table(consoliado_pagos_brad2,index='PAGOSVENCIDOS',values=['monto_promesa'], aggfunc = ['count',np.sum], margins=True,dropna=False)
prom_brad_div_mes = pd.DataFrame(prom_brad_div_mes.to_records())
prom_brad_div_mes.columns = ['Division','NumeroPromesas','SumaPromesas']
prom_brad_div_mes.fillna(0,inplace=True)
prom_brad_div_mes

,Division,NumeroPromesas,SumaPromesas
0,A,33,61077.00
1,B,6,10201.00
2,C0 4PV,285,458991.09
3,C0 5PV,129,231211.00
4,C0 6PV,101,187317.00
5,C1,142,233237.00
6,C2,136,204871.00
7,C3,100,131429.00
8,C4,45,44730.00
9,All,977,1563064.09


In [76]:
#Obtenemos las promesas restantes a partir del día de hoy
restantes_brad = consoliado_pagos_brad2.loc[consoliado_pagos_brad2['fecha_promesa']>=ayer]
promesas_restantes_brad = pd.pivot_table(restantes_brad,index=['PAGOSVENCIDOS'],values=['monto_promesa'],aggfunc=[np.sum],dropna=False)
promesas_restantes_brad = pd.DataFrame(promesas_restantes_brad.to_records())
promesas_restantes_brad.columns = ['Division','PromesasRestantes']
promesas_restantes_brad.fillna(0,inplace=True)
promesas_restantes_brad

,Division,PromesasRestantes
0,A,7000.0
1,C0 4PV,38314.0
2,C0 5PV,21077.0
3,C0 6PV,5993.0
4,C1,17863.0
5,C2,20832.0
6,C3,1250.0
7,C4,700.0


In [79]:
pagos_totales_brad2 = pd.pivot_table(pagos_brad,index=['TIPO_ASIGANCION'],values=['MONTO','Facturacion'],aggfunc=['count',np.sum],dropna=False)
pagos_totales_brad2 = pd.DataFrame(pagos_totales_brad2.to_records())
pagos_totales_brad2 = pagos_totales_brad2.iloc[:,[0,1,4,3]]
pagos_totales_brad2.columns = ['Division','NumeroPagos','SumaPagos','SumaFacturacion']
pagos_totales_brad2.fillna(0,inplace=True)
pagos_totales_brad2['FechaFact'] = max(pagos_brad['FECHA_PAGO'])

In [80]:
final_brad = pd.merge(pagos_totales_brad2,prom_brad_div_mes,how='left',on=['Division'])
final_brad = pd.merge(final_brad,promesas_restantes_brad,how='left',on=['Division'])
final_brad.fillna(0,inplace=True)
final_brad.insert(loc=0,column='Cliente',value='Bradesco')
final_brad

,Cliente,Division,NumeroPagos,SumaPagos,SumaFacturacion,FechaFact,NumeroPromesas,SumaPromesas,PromesasRestantes
0,Bradesco,A,16,33962.00,9169.7400,2021-01-15,33,61077.00,7000.0
1,Bradesco,B,5,25364.00,6341.0000,2021-01-15,6,10201.00,0.0
2,Bradesco,C0 4PV,224,325377.32,22776.4124,2021-01-15,285,458991.09,38314.0
3,Bradesco,C0 5PV,106,132900.06,11961.0054,2021-01-15,129,231211.00,21077.0
4,Bradesco,C0 6PV,76,117564.50,11756.4500,2021-01-15,101,187317.00,5993.0
5,Bradesco,C1,104,183434.00,31183.7800,2021-01-15,142,233237.00,17863.0
6,Bradesco,C2,77,139184.84,26445.1196,2021-01-15,136,204871.00,20832.0
7,Bradesco,C3,52,66521.00,13969.4100,2021-01-15,100,131429.00,1250.0
8,Bradesco,C4,18,16503.00,3795.6900,2021-01-15,45,44730.00,700.0


# Reporte Banco Azteca  

In [81]:
# #Hacemos la consulta referente a las gestiones de Banco Azteca
# sql_gest_baz = cursor.callproc('baz_rpt_gestiones_detallado', [mes,today])
# for result in cursor.stored_results():
#      gestion_baz = pd.DataFrame(result.fetchall())
# gestion_baz.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion_resultado','fecha_promesa','monto_promesa','comentarios']
# gestion_baz = gestion_baz.loc[gestion_baz['dictamen']=='PROMESA']
# gestion_baz['monto_promesa'] = pd.to_numeric(gestion_baz['monto_promesa'])
# gestion_baz = gestion_baz.loc[gestion_baz['fecha_promesa'].map(str).str[:4].map(int)<2100]
# gestion_baz['fecha_promesa'] = pd.to_datetime(gestion_baz['fecha_promesa'],dayfirst=False)
# #gestion_baz.head(3)

In [82]:
# #Obtenemos datos de las promesas de Banco Azteca
# gestion_baz = gestion_baz.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
# prom_baz_div_mes = pd.pivot_table(gestion_baz,index=['dictamen'],values=['unegocio_id','monto_promesa'],aggfunc=('count',np.sum))
# prom_baz_div_mes = pd.DataFrame(prom_baz_div_mes.to_records())
# prom_baz_div_mes = prom_baz_div_mes.iloc[:,[0,1,2]]
# prom_baz_div_mes.columns = ['Division','NumeroPromesas','SumaPromesas']
# prom_baz_div_mes.fillna(0,inplace=True)
# prom_baz_div_mes = prom_baz_div_mes.replace('PROMESA','COBRANZA')
# prom_baz_div_mes

In [83]:
# #Obtenemos las promesas restantes a partir del día de hoy
# restantes_baz = gestion_baz.loc[gestion_baz['fecha_promesa']>=ayer]

# promesas_restantes_baz = pd.pivot_table(restantes_baz,index=['dictamen'],values=['monto_promesa'],aggfunc=[np.sum],dropna=False)
# promesas_restantes_baz = pd.DataFrame(promesas_restantes_baz.to_records())
# promesas_restantes_baz.columns = ['Division','PromesasRestantes']
# promesas_restantes_baz = promesas_restantes_baz.replace('PROMESA','COBRANZA')
# promesas_restantes_baz.fillna(0,inplace=True)
# promesas_restantes_baz

In [84]:
# pagos_baz2['division'] = 'COBRANZA'
# pagos_baz2['FNFACTURACION'] = [x*.2 for x in pagos_baz2['FNMONTOPAGADO']]
# pagos_totales_baz = pd.pivot_table(pagos_baz2,index=['division'],values=['FNMONTOPAGADO'],aggfunc=['count',np.sum],dropna=False)
# pagos_totales_baz = pd.DataFrame(pagos_totales_baz.to_records())
# #pagos_totales_baz = pagos_totales_baz.iloc[:,[0,1,4,3]]
# pagos_totales_baz.columns = ['Division','NumeroPagos','SumaPagos']
# #pagos_totales_baz.fillna(0,inplace=True)
# #pagos_totales_baz['FechaFact'] = ayer
# pagos_totales_baz

In [85]:
# fact_baz['Total Recuperado'] = np.where(fact_baz['Inicio de Semana']<mes3,fact_baz['Total Recuperado']*.85,fact_baz['Total Recuperado'])
# fact_baz['Total Comisión'] = np.where(fact_baz['Inicio de Semana']<mes3,fact_baz['Total Comisión']*.85,fact_baz['Total Comisión'])

In [86]:
# #Hacemos la consulta de pagos y facturacion
# fact_baz['Division2'] = 'COBRANZA'
# pagos_fact_baz = pd.pivot_table(fact_baz,index=['Division2'],values=['Total Recuperado','Total Comisión'],aggfunc=[np.sum])
# pagos_fact_baz = to_dataframe(pagos_fact_baz)
# pagos_fact_baz = pagos_fact_baz.iloc[:,[0,2,1]]
# pagos_fact_baz.columns = ['Division','SumaPagos','SumaFacturacion']
# pagos_fact_baz['FechaFact'] = max(fact_baz['Final de Semana'])
# pagos_fact_baz

In [87]:
# final_baz = pd.merge(pagos_fact_baz,prom_baz_div_mes,how='left',on=['Division'])
# final_baz = pd.merge(final_baz,promesas_restantes_baz,how='left',on=['Division'])
# final_baz.fillna(0,inplace=True)
# final_baz.insert(loc=0,column='Cliente',value='Banco Azteca')
# final_baz

# Reporte Credifiel

In [88]:
#Hacemos la consulta referente a las gestiones de Credifiel
gestion_cred = cs.query_naucalpan('credifiel_rpt_gestiones_detallado',mes,today)
gestion_cred.columns = ['folio_gestion','unegocio_id','fecha_gestion','hora_gestion','credito','nombre_credito','telefono','usuario','nombre_usuario','accion','resultado','fecha_promesa','monto_promesa','comentarios','accion_credifiel','resultado_credifiel','dictamen']
gestion_cred['monto_promesa'] = pd.to_numeric(gestion_cred['monto_promesa'])
gestion_cred = gestion_cred.loc[gestion_cred['accion_credifiel']=='PDP']
gestion_cred['credito'] = [x[3:] for x in gestion_cred['credito']]
gestion_cred['credito'] = pd.to_numeric(gestion_cred['credito'])
#gestion_cred.head(3)

In [89]:
#Hacemos la consulta de la asignacion
asignacion_cred = pd.read_excel('/home/estadistico/Documents/Erick/Credifiel/Asignacion/Asignacion Credifiel '+mes2+'.xlsx')
asignacion_cred = asignacion_cred.drop_duplicates(subset=['CREDITO','SALDO TOTAL REAL'])
asignacion_cred['CREDITO'] = pd.to_numeric(asignacion_cred['CREDITO'])

In [90]:
#Hacemos el consolidado de la asignacion y promesado
consolidado_cred = pd.merge(gestion_cred,asignacion_cred,how='left',left_on=['credito'],right_on=['CREDITO'])
cols = ['monto_promesa','unegocio_id']
consolidado_cred[cols] = consolidado_cred[cols].apply(pd.to_numeric, errors='coerce')
consolidado_cred = consolidado_cred.loc[consolidado_cred['monto_promesa']>50]
consolidado_cred = consolidado_cred[(consolidado_cred['monto_promesa']<500000)]
consolidado_cred['fecha_promesa'] = pd.to_datetime(consolidado_cred['fecha_promesa'],dayfirst=False)
consolidado_cred.replace('CONTACTO','EXTRA',inplace=True)

In [91]:
pagos_cred3 = pd.merge(pagos_cred2,consolidado_cred[['credito','nombre_usuario','fecha_promesa','monto_promesa','ARANCEL']],how='left',left_on=['ID'],right_on=['credito'])
pagos_cred3['ARANCEL'].fillna(np.mean(pagos_cred3['ARANCEL']),inplace=True)
pagos_cred3['PagoFacturacion'] = [x*y for x,y in zip(pagos_cred3['ARANCEL'],pagos_cred3['IMPORTE'])]
pagos_cred3['division'] = 'EXTRA'
pagos_cred3 = pagos_cred3.drop_duplicates(subset=['ID','IMPORTE'])

In [92]:
#Obtenemos datos de las promesas de Bradesco
prom_cred_div_mes = pd.pivot_table(consolidado_cred,index=['dictamen'],values=['monto_promesa'], aggfunc=['count',np.sum], dropna=False)
prom_cred_div_mes = pd.DataFrame(prom_cred_div_mes.to_records())
#prom_cred_div_mes = prom_cred_div_mes.iloc[:,[0,4,8,6,5]]
prom_cred_div_mes.columns = ['Division','NumeroPromesas','SumaPromesas']
#prom_cred_div_mes = prom_cred_div_mes.sort_values(by=['SumaPago'],ascending =False)
#prom_cred_div_mes = prom_cred_div_mes.reset_index(drop=True)
#prom_cred_div_mes.fillna(0,inplace=True)
#prom_brad_div_mes[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = prom_brad_gestor_mes[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
prom_cred_div_mes

,Division,NumeroPromesas,SumaPromesas
0,EXTRA,86,257731.16


In [93]:
#Obtenemos las promesas restantes a partir del día de hoy
restantes_cred = consolidado_cred.loc[consolidado_cred['fecha_promesa']>=ayer]

promesas_restantes_cred = pd.pivot_table(restantes_cred,index=['dictamen'],values=['monto_promesa'],aggfunc=[np.sum],dropna=False)
promesas_restantes_cred = pd.DataFrame(promesas_restantes_cred.to_records())
#promesas_restantes_brad = promesas_restantes_brad.iloc[:,[0,3,8,6,5]]
promesas_restantes_cred.columns = ['Division','PromesasRestantes']
promesas_restantes_cred.fillna(0,inplace=True)

#posible cambio
#promesas_restantes_brad = promesas_restantes_brad.sort_values(by='SumaPagos',ascending=False)
#promesas_restantes_brad[['SumaPromesas']] = promesas_restantes_brad[['SumaPromesas']].applymap("{0:.2f}".format)
promesas_restantes_cred

,Division,PromesasRestantes
0,EXTRA,88237.33


In [94]:
pagos_totales_cred = pd.pivot_table(pagos_cred3,index=['division'],values=['IMPORTE','PagoFacturacion'],aggfunc=['count',np.sum],dropna=False)
pagos_totales_cred = pd.DataFrame(pagos_totales_cred.to_records())
pagos_totales_cred = pagos_totales_cred.iloc[:,[0,1,3,4]]
pagos_totales_cred.columns = ['Division','NumeroPagos','SumaPagos','SumaFacturacion']
pagos_totales_cred.fillna(0,inplace=True)
#pagos_totales_cred[['SumaPagos','SumaFacturacion']] = pagos_totales_cred[['SumaPagos','SumaFacturacion']].applymap("${0:,.2f}".format)
pagos_totales_cred['FechaFact'] = max(pagos_cred3['FECHA'])
pagos_totales_cred

,Division,NumeroPagos,SumaPagos,SumaFacturacion,FechaFact
0,EXTRA,34,183076.96,40253.956961,2021-01-14


In [95]:
final_cred = pd.merge(pagos_totales_cred,prom_cred_div_mes,how='left',on=['Division'])
final_cred = pd.merge(final_cred,promesas_restantes_cred,how='left',on=['Division'])
final_cred.fillna(0,inplace=True)
final_cred.insert(loc=0,column='Cliente',value='Credifiel')
final_cred = final_cred.replace('EXTRA','COBRANZA')
final_cred

,Cliente,Division,NumeroPagos,SumaPagos,SumaFacturacion,FechaFact,NumeroPromesas,SumaPromesas,PromesasRestantes
0,Credifiel,COBRANZA,34,183076.96,40253.956961,2021-01-14,86,257731.16,88237.33


# Reporte final Naucalpan

In [96]:
final_naucalpan = pd.concat([final_liv,final_brad],ignore_index=True)
final_naucalpan = pd.concat([final_naucalpan,final_cred],ignore_index=True)
#final_naucalpan = pd.concat([final_naucalpan,final_baz],ignore_index=True)
final_naucalpan.insert(loc=0,column='Sucursal',value='Naucalpan')
final_naucalpan.insert(loc=0,column='Pais',value='Mexico')
final_naucalpan['TendenciaCierre1'] = [(30/x.day)*y for x,y in zip(final_naucalpan['FechaFact'],final_naucalpan['SumaFacturacion'])]
final_naucalpan = final_naucalpan.replace('EXTRA','EXTRAJUDICIAL')
final_naucalpan = final_naucalpan.replace('COBRANZA','EXTRAJUDICIAL')
final_naucalpan.fillna(0,inplace=True)
final_naucalpan

,Pais,Sucursal,Cliente,Division,NumeroPagos,SumaPagos,SumaFacturacion,FechaFact,NumeroPromesas,SumaPromesas,PromesasRestantes,TendenciaCierre1
0,Mexico,Naucalpan,Liverpool,DKN00REM,103,399556.00,111875.680000,2021-01-17,179,749282.74,192472.87,197427.670588
1,Mexico,Naucalpan,Liverpool,DKN02REM,97,357302.99,142921.196000,2021-01-17,152,669093.44,151329.00,252213.875294
2,Mexico,Naucalpan,Liverpool,DKN04REM,50,180155.37,90077.685000,2021-01-17,78,261983.37,72174.00,158960.620588
3,Mexico,Naucalpan,Liverpool,SA001REM,107,153899.98,23084.997000,2021-01-17,170,272277.24,81950.10,40738.230000
4,Mexico,Naucalpan,Liverpool,SA002REM,9,16701.00,2672.160000,2021-01-17,10,21957.00,0.00,4715.576471
5,Mexico,Naucalpan,Bradesco,A,16,33962.00,9169.740000,2021-01-15,33,61077.00,7000.00,18339.480000
6,Mexico,Naucalpan,Bradesco,B,5,25364.00,6341.000000,2021-01-15,6,10201.00,0.00,12682.000000
7,Mexico,Naucalpan,Bradesco,C0 4PV,224,325377.32,22776.412400,2021-01-15,285,458991.09,38314.00,45552.824800
8,Mexico,Naucalpan,Bradesco,C0 5PV,106,132900.06,11961.005400,2021-01-15,129,231211.00,21077.00,23922.010800
9,Mexico,Naucalpan,Bradesco,C0 6PV,76,117564.50,11756.450000,2021-01-15,101,187317.00,5993.00,23512.900000


# Pagos sin promesa asignada

In [103]:
#Pagos de bradesco que no tienen promesa asignada
gestion_brad_mes2 = gestion_brad_mes.drop_duplicates(subset='NUMERO_TARJETA')
pagos_noasig_brad = pd.merge(pagos_brad,gestion_brad_mes2[['NUMERO_TARJETA','nombre_credito']],how='left',on='NUMERO_TARJETA')
pagos_noasig_brad = pagos_noasig_brad.loc[pagos_noasig_brad['nombre_credito'].isnull()]
pagos_noasig_brad_pivot = pd.pivot_table(pagos_noasig_brad,index=['TIPO_ASIGANCION'],values=['MONTO'],aggfunc=[np.sum,'count'])
pagos_noasig_brad_pivot = pd.DataFrame(pagos_noasig_brad_pivot.to_records())
pagos_noasig_brad_pivot.columns = ['Division','RecuperadoNoPromesa','PagosNoPromesa']
pagos_noasig_brad_pivot

,Division,RecuperadoNoPromesa,PagosNoPromesa
0,A,3720.00,1
1,B,21817.00,2
2,C0 4PV,102219.32,85
3,C0 5PV,56335.03,48
4,C0 6PV,36487.50,31
5,C1,81754.00,52
6,C2,37482.84,26
7,C3,3342.00,5
8,C4,640.00,1


In [105]:
#Pagos de Liverpool que no tienen promesa asignada
gestion_liv_mes2 = gestion_liv_mes.drop_duplicates(subset='credito')
pagos_noasig_liv = pd.merge(pagos_liv,gestion_liv_mes2[['credito','nombre_credito']],left_on='Cuenta',right_on='credito',how='left')
pagos_noasig_liv = pagos_noasig_liv.loc[pagos_noasig_liv['nombre_credito'].isnull()]
pagos_noasig_liv_pivot = pd.pivot_table(pagos_noasig_liv,index=['Id'],values=['Monto'],aggfunc=[np.sum,'count'])
pagos_noasig_liv_pivot = pd.DataFrame(pagos_noasig_liv_pivot.to_records())
pagos_noasig_liv_pivot.columns = ['Division','RecuperadoNoPromesa','PagosNoPromesa']
pagos_noasig_liv_pivot

,Division,RecuperadoNoPromesa,PagosNoPromesa
0,DKN00REM,67649.00,21
1,DKN02REM,64820.99,36
2,DKN04REM,28053.00,8
3,SA001REM,25609.83,28
4,SA002REM,4305.00,4


In [106]:
#Pagos de Liverpool que no tienen promesa asignada
gestion_cred2 = gestion_cred.drop_duplicates(subset='credito')
pagos_noasig_cred = pd.merge(pagos_cred2,gestion_cred2[['credito','nombre_credito']],left_on='ID',right_on='credito',how='left')
pagos_noasig_cred = pagos_noasig_cred.loc[pagos_noasig_cred['nombre_credito'].isnull()]
pagos_noasig_cred['Cliente'] = 'EXTRAJUDICIAL'
pagos_noasig_cred_pivot = pd.pivot_table(pagos_noasig_cred,index=['Cliente'],values=['IMPORTE'],aggfunc=[np.sum,'count'])
pagos_noasig_cred_pivot = pd.DataFrame(pagos_noasig_cred_pivot.to_records())
pagos_noasig_cred_pivot.columns = ['Division','RecuperadoNoPromesa','PagosNoPromesa']
pagos_noasig_cred_pivot

,Division,RecuperadoNoPromesa,PagosNoPromesa
0,EXTRAJUDICIAL,88441.0,9


In [107]:
list1 = [pagos_noasig_cred_pivot,pagos_noasig_liv_pivot,pagos_noasig_brad_pivot]
pagos_noasig_naucalpan = pd.concat(list1,ignore_index=True)
pagos_noasig_naucalpan

,Division,RecuperadoNoPromesa,PagosNoPromesa
0,EXTRAJUDICIAL,88441.00,9
1,DKN00REM,67649.00,21
2,DKN02REM,64820.99,36
3,DKN04REM,28053.00,8
4,SA001REM,25609.83,28
5,SA002REM,4305.00,4
6,A,3720.00,1
7,B,21817.00,2
8,C0 4PV,102219.32,85
9,C0 5PV,56335.03,48


In [108]:
final = pd.merge(final_naucalpan,pagos_noasig_naucalpan,on='Division',how='left')
final

,Pais,Sucursal,Cliente,Division,NumeroPagos,SumaPagos,SumaFacturacion,FechaFact,NumeroPromesas,SumaPromesas,PromesasRestantes,TendenciaCierre1,RecuperadoNoPromesa,PagosNoPromesa
0,Mexico,Naucalpan,Liverpool,DKN00REM,103,399556.00,111875.680000,2021-01-17,179,749282.74,192472.87,197427.670588,67649.00,21
1,Mexico,Naucalpan,Liverpool,DKN02REM,97,357302.99,142921.196000,2021-01-17,152,669093.44,151329.00,252213.875294,64820.99,36
2,Mexico,Naucalpan,Liverpool,DKN04REM,50,180155.37,90077.685000,2021-01-17,78,261983.37,72174.00,158960.620588,28053.00,8
3,Mexico,Naucalpan,Liverpool,SA001REM,107,153899.98,23084.997000,2021-01-17,170,272277.24,81950.10,40738.230000,25609.83,28
4,Mexico,Naucalpan,Liverpool,SA002REM,9,16701.00,2672.160000,2021-01-17,10,21957.00,0.00,4715.576471,4305.00,4
5,Mexico,Naucalpan,Bradesco,A,16,33962.00,9169.740000,2021-01-15,33,61077.00,7000.00,18339.480000,3720.00,1
6,Mexico,Naucalpan,Bradesco,B,5,25364.00,6341.000000,2021-01-15,6,10201.00,0.00,12682.000000,21817.00,2
7,Mexico,Naucalpan,Bradesco,C0 4PV,224,325377.32,22776.412400,2021-01-15,285,458991.09,38314.00,45552.824800,102219.32,85
8,Mexico,Naucalpan,Bradesco,C0 5PV,106,132900.06,11961.005400,2021-01-15,129,231211.00,21077.00,23922.010800,56335.03,48
9,Mexico,Naucalpan,Bradesco,C0 6PV,76,117564.50,11756.450000,2021-01-15,101,187317.00,5993.00,23512.900000,36487.50,31


In [109]:
final.to_csv(''+ruta+'/'+mes2+'/Reportes diarios/Reporte sucursal/Reporte sucursal '+today2+'.csv',index=False,header=True)